# Introduction
Before starting this tutorial, you'll need to have at least one sequence from the Boreas dataset downloaded.
If you're working on a local machine, follow these steps to download a sequence:
1. [Create an AWS account](https://aws.amazon.com/premiumsupport/knowledge-center/create-and-activate-aws-account/)
2. [Install the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html)
3. Create a `root` folder to store the dataset, example: `/path/to/data/boreas/` Each sequence will then be a folder under `root`.
4. Use the AWS CLI to download a sequence:
```
root=/path/to/data/boreas/
sequence=boreas-2021-09-02-11-42
aws s3 sync s3://boreas/$sequence $root$sequence
```

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from pyboreas import BoreasDataset
from pyboreas.utils.utils import get_inverse_tf

root = '/mnt/ssd_external/radar_data/boreas/'
split = None
# AWS: Note: Free Tier SageMaker instances don't have enough storage (25 GB) for 1 sequence (100 GB)
# root = '/home/ec2-user/SageMaker/boreas/'
# split = [['boreas-2021-09-02-11-42', 163059759e6, 163059760e6-1]]

# With verbose=True, the following will print information about each sequence
bd = BoreasDataset(root, split=split, verbose=True)
# Grab the first sequence
seq = bd.sequences[0]

ModuleNotFoundError: No module named 'pyboreas'

In [3]:
# Each sequence has it's own calibration:
seq.calib.print_calibration()

NameError: name 'seq' is not defined

In [ ]:
%matplotlib inline
# Let's visualize a lidar frame:
lid = seq.get_lidar(0)
lid.passthrough([-40, 40, -40, 40, -10, 30])
lid.visualize(figsize=(10, 10), color='intensity', vmin=5, vmax=40)

In [ ]:
# Now let's visualize the first camera frame:
cam = seq.get_camera(0)
cam.visualize(figsize=(10, 10))

In [ ]:
# Now let's visualize the first radar frame:
rad = seq.get_radar(0)
rad.visualize(cart_resolution=0.25, cart_pixel_width=1000)

In [ ]:
# Note that each sensor frame has a timestamp, pose (4x4 homogeneous transform), and velocity information.
lid = seq.get_lidar(0)
print('Lidar:')
print('timestamp: {}'.format(lid.timestamp))
print('pose (T_enu_lidar):')
print(lid.pose)
print('velocity (wrt ENU):')
print(lid.velocity)
print('body rate (wrt sensor):')
print(lid.body_rate)

# Note that lidar and camera frames are collected at 10Hz, but radar frames collected at 4 Hz.

In [ ]:
# To transform data from one frame to another, use the poses
index = 0
lid = seq.get_lidar(index)
p_lid = np.array([1, 0, 0, 1]).reshape(4, 1)
print('point in lidar frame')
print(p_lid)
print('lidar pose (T_enu_lidar) at time: {}'.format(lid.timestamp))
T_enu_lidar = lid.pose
print(T_enu_lidar)

# **Important: camera, lidar, radar measurements are NOT synchronous
# camera frame X does not necessarily correspond to lidar frame X
# They may be close, but will have been collected at different times, see below:
cam = seq.get_camera(index)
print('camera pose (T_enu_camera) at time: {}'.format(cam.timestamp))
T_enu_camera = cam.pose
print(T_enu_camera)

T_camera_lidar = np.matmul(get_inverse_tf(T_enu_camera), T_enu_lidar)
print('T_camera_lidar:')
print(T_camera_lidar)
print('distance: {}'.format(np.linalg.norm(T_camera_lidar[:3, 3])))

print('point in camera frame:')
p_cam = np.matmul(T_camera_lidar, p_lid)
print(p_cam)

In [ ]:
# Example: using an iterator
cam_iter = bd.sequences[0].get_camera_iter()
cam0 = next(cam_iter)  # First camera frame
cam1 = next(cam_iter)  # Second camera frame
print(cam0.timestamp)
print(cam1.timestamp)